In [2]:
from manipulability_Optimizer import *
from withoutUpdate import gradientAscent_no_update
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Maximize Manipulability : How To Use
***
#### 1. Call `gradientAscent()` to have a plain unconstrained Optimizer on the simplest form of the Manipulability Measure (only Singularities).
#### 2. Add arguments according to needs:
- **initialPose** = "upwards", "downwards". If None, it starts with a totally arbitrary joint config $q_{init}$ (within the joint limits).

Extra : You can also input a specified configuration vector of 7 angles in radians within the joint limits

- **constrainedDimensions** = [x,y,z,$\alpha, \beta, \gamma$] . Give a 1 to the dimensions you want to block, a 0 to the ones that should be free of constraint.

- **lr** = Learning Rate of Gradient Ascent. Give it any value above 0.0 .

- **c** = Convergence Rate. The smaller it is, the longer the algorithm is trying to maximize.

- **P** = True or False. To include the Joint Limit Penalty in the Manipulability Measure.

- **k** = Parameter that influences how the joint limits are impacting overall Manipulability Measure. Give it a value above 0. Under 10.000, the search evolves very slow.


## Values by default:
`initialPose = None, 
constrainedDimensions = [0,0,0,0,0,0], 
lr = 6.5, 
c = 0.0001, 
P = False, 
k = 2000`

# 1. Hyperparameters Tuning

## a) Learning Rate $r$

#### i) Free Optimization

In [ ]:
wTotal = []
learningRates = [1, 2, 4, 6, 7, 8, 10]
for i in range(9):
    q = []
    for j in range(7):
        randAngle = rnd.uniform(robot.get_joint_limits()[j]['lower'],robot.get_joint_limits()[j]['upper'])
        q.append(randAngle)
    w = []
    print(f"Initial Configuration : {q}")
    for lr in learningRates:
        #200 iterations/run
        result = gradientAscent(lr=lr, initialPose=q)
        print(f"Manipulability reached with lr = {lr}  : ", result[-1].item())
        w.append(result)
    wTotal.append(w)      

In [ ]:
x = np.arange(0,200)

titles = [r"Random $q_{init_1}$", r"Random $q_{init_2}$", r"Random $q_{init_3}$", r"Random $q_{init_4}$"]
fig, axs = plt.subplots(2, 2, figsize=(14, 8), constrained_layout=True)
for ax, i in zip (axs.flat, range(4)):
    for j, val in enumerate(cm.jet(np.linspace(0,1,7))):
        ax.plot(x,wTotal[i][j], color=val, label=f'{learningRates[j]}')
    
    ax.grid()
    ax.legend()
    ax.set_ylim(0.12,0.133)
    ax.set_title('%s' % titles[i], fontsize=16)
    ax.set_ylabel('Manipulability (w)')
    ax.set_xlabel('# Iterations')
    fig.savefig("diffLR1.png", dpi=500)

#### ii) Constrained Optimization (Hand-Pose + Joint Limits)

In [ ]:
# Here, some components of the source code are reused (only) for the sake of visualization
def randomPoseConstrained(desiredPose):
    robotKDL = URDF.from_parameter_server()
    kdl_kin = KDLKinematics(robotKDL, "panda_link0", "palm")

    motherQ = [-2.497678279876709, 1.2272013425827026, 2.727571487426758, -2.5281753540039062, 0.3514966368675232, 1.649809718132019, -0.8299999833106995]
    #motherQ = [-2.0, 0.0, 2.3259260654449463, -2.4263031482696533, -2.0412862300872803, 0.8505620360374451, -0.8299999833106995]
    t_base_Palm = kdl_kin.forward(motherQ)
    
    q = randomConfig()
    
    isNan = True
    while isNan :
        rot = randomRotation(desiredPose, t_base_Palm)
        rotTrans = randomTranslation(rot, q)

        temp = ik_franka(
            #indicates the pose of the EE to calculate IK from
            desired_transform = rotTrans,
            q7=-0.83, #q0,q1,q2,q3,q4,q5,q6 --> 7-dim Vector ;;;; q7 is joint 8 --> wrist
            q_actual= motherQ)
        temp = [temp.solution_1, temp.solution_2, temp.solution_3, temp.solution_4]
        for solu in temp:
            if not np.isnan(solu).any():
                simulatorView(solu)
                isNan = False
                joint_init = solu
                break
        q = randomConfig()
                
    return joint_init

In [ ]:
wTotal1 = []
learningRates = [1, 2, 4, 6, 7, 8, 10]
for i in range(4):
    q = list(randomPoseConstrained("upwards"))
    w = []
    print(f"Initial Configuration : {q}")
    for lr in learningRates:
        #200 iterations/run
        # Gradient Ascent is run with hand-tilting blocked, randomly upwards hand  initialization 
        # and joint penalty P enabled
        result = gradientAscent(initialPose=q, constrainedDimensions = [0,0,0,1,1,0], lr=lr, P=True)
        print(f"Manipulability reached with lr = {lr}  : ", result[-1].item())
        w.append(result)
    wTotal1.append(w)

In [ ]:
x = np.arange(0,200)

titles = [r"Random $q_{init_1}$ (hand facing upwards)", r"Random $q_{init_2}$ (hand facing upwards)", r"Random $q_{init_3}$ (hand facing upwards)", r"Random $q_{init_4}$ (hand facing upwards)"]
fig, axs = plt.subplots(2, 2, figsize=(14, 8), constrained_layout=True)
for ax, i in zip (axs.flat, range(4)):
    for j, val in enumerate(cm.jet(np.linspace(0.,0.6,7))):
        ax.plot(x,wTotal1[i][j], color=val, label=f'{learningRates[j]}')
    
    ax.grid()
    ax.legend()
    ax.set_ylim(0.09,0.133)
    ax.set_title('%s' % titles[i], fontsize=16)
    ax.set_ylabel('Manipulability (w*P)')
    ax.set_xlabel('# Iterations')
    fig.savefig("diffLRConstrained.png", dpi=500)

## b) Convergence Treshold $c$

## c) Joint Limits weight $k$

In [ ]:
#The lower, the slower the convergence, but the more importance to joint limits.
# The highern the quickest the convegence, but the less importane to joint limits.

### d) Patterns in High Manipubility (Search Space and Configuration Space)

In [ ]:
configs = []
space = []
manips = []
o = torch.zeros(9)
for i in range(30000):
    
    q = torch.cat((torch.tensor(randomConfig()), torch.tensor([0.,0.])))
    m = manipulability(q, o, o, o, 10000)
    s = forwardKinematics(q)[0]
    
    configs.append(q)
    manips.append(m)
    space.append(s)
    if (i+1)%100 == 0:
        print(f"Reached {i+1}-th sample")
    poses_fk = fk_rbo_hand(index_airmass = [0,0],
                            middle_airmass = [0,0],
                               ring_airmass = [0,0],
                               little_airmass = [0,0],
                               thumb_airmass = [0,0.0,0.0,0.0],
                               palm_airmass = [0.],
                               in_palm_frame = 1,
                               scaled_masses = 1,
                               panda_joint_angles=q[:7])          

In [ ]:
spaceTrans = torch.transpose(torch.stack(space),0,1)
print(spaceTrans[2])
mans = torch.stack(manips)
print(mans)

In [ ]:
x = spaceTrans[0]
y = spaceTrans[1]
z = spaceTrans[2]
newX = []
newY = []
newZ = []
newMans = []

for i in range(30000):
    if  mans[i] > 0.12 :
        newX.append(x[i])
        newY.append(y[i])
        newZ.append(z[i])
        newMans.append(mans[i])
        
      
        
        

In [ ]:
# On 30.000 random free of constraint random init, we reached less than 0.5% of the time a higher than 90% manipulability-score.
# On 30.000, 50 highest, we check the distribution of joint values. We notice that the 2 key joints are offset elbow and offset wrist
# Do the same with joint limits !!!!! Compare how the joints will be in the middle always !!! 
newConfigs = []
for i in range(30000):
    if  (mans[i] > 0.026) :
        newConfigs.append(configs[i])
print(len(newConfigs))


In [ ]:
# 3D Heatmap in Python using matplotlib
%matplotlib inline
  
# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from pylab import *
  
# creating a dummy dataset
x = spaceTrans[0]
y = spaceTrans[1]
z = spaceTrans[2]
  
# creating 3d figures
fig = plt.figure(figsize=(10, 10))
ax = Axes3D(fig)
  
# creating the heatmap
img = ax.scatter(newX, newY, newZ, marker='.',
                 s=100, c='blue', alpha=0.6)

  
# adding title and labels
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

ax.axes.set_zlim3d(bottom=-0.4, top=1.3) 
ax.axes.set_ylim3d(bottom=-0.75, top=0.75) 

ax.view_init(elev=0, azim=90)
# displaying plot
plt.show()
fig.savefig("mushroomSliceVerti.png", dpi=500)

In [ ]:
finConfig = torch.transpose(torch.stack(newConfigs),0,1)
print(finConfig)

In [ ]:
from scipy.stats.kde import gaussian_kde
from numpy import linspace
# create fake data
data = finConfig[6]
# this create the kernel, given an array it will estimate the probability over that values
kde = gaussian_kde( data )
# these are the values over wich your kernel will be evaluated
dist_space = linspace( min(data), max(data), 100 )
# plot the results
plt.xlim(-3,3)
plt.grid()
plt.xlabel("Joint 6 (Angle in Radians)")
plt.ylabel("Density")
plt.plot( dist_space, kde(dist_space), color="m" )

In [ ]:
configs = []
for i in range(9):
    configs.append(randomPoseConstrained("downwards"))
    

In [ ]:
for q,i in zip(configs, range(len(configs))):
    print(i," : ", q)
    poses_fk = fk_rbo_hand(index_airmass = [0,0],
                            middle_airmass = [0,0],
                               ring_airmass = [0,0],
                               little_airmass = [0,0],
                               thumb_airmass = [0,0.0,0.0,0.0],
                               palm_airmass = [0.],
                               in_palm_frame = 1,
                               scaled_masses = 1,
                               panda_joint_angles=q[:7])
    time.sleep(15)

# Non-updated constraint vs. Updated constraint  

In [ ]:
q = list(randomPoseConstrained("upwards"))

### 1. Non-updated

In [ ]:
gradientAscent_no_update(initialPose=q, constrainedDimensions=[0,0,0,1,1,0])

### 2. Updated

In [ ]:
poses_fk = fk_rbo_hand(index_airmass = [0,0],
                            middle_airmass = [0,0],
                               ring_airmass = [0,0],
                               little_airmass = [0,0],
                               thumb_airmass = [0,0.0,0.0,0.0],
                               palm_airmass = [0.],
                               in_palm_frame = 1,
                               scaled_masses = 1,
                               panda_joint_angles=q[:7])

In [ ]:
gradientAscent(initialPose=q, constrainedDimensions=[0,0,0,1,1,0]) #code has been changed for this experiment

# Without vs. With Joint Limits

### Without

In [ ]:
gradientAscent(initialPose=q)

### With

In [ ]:
poses_fk = fk_rbo_hand(index_airmass = [0,0],
                            middle_airmass = [0,0],
                               ring_airmass = [0,0],
                               little_airmass = [0,0],
                               thumb_airmass = [0,0.0,0.0,0.0],
                               palm_airmass = [0.],
                               in_palm_frame = 1,
                               scaled_masses = 1,
                               panda_joint_angles=q[:7])

In [ ]:
gradientAscent(initialPose=q, P=True)

# Plot severities of constraints for 4 random initial configuration

In [ ]:
wTotal = []
#Nothing blocked
#Z is blocked
#All trans is blocked
#rot around x,y is blocked (alpha, beta)
# All rot is blocked
# Everything blocked

# We expect the manipulability to reach a lower global maximum in this order (higher manip first, lowest manip last)

blocked = [[0,0,0,0,0,0],[0,0,1,0,0,0], [1,1,1,0,0,0], [0,0,0,1,1,0], [0,0,0,1,1,1], [1,1,1,1,1,1]]
for i in range(4):
    q = list(randomPoseConstrained("upwards"))
    w = []
    print(f"Initial Configuration : {q}")
    for blocked_dim in blocked:
        #200 iterations/run
        result = gradientAscent(constrainedDimensions=blocked_dim, initialPose=q)
        print(f"Manipulability reached with blocked dimensions  = {blocked_dim}  : ", result[-1].item())
        w.append(result)
    wTotal.append(w) 

In [ ]:
x = np.arange(0,200)

titles = [r"Random $q_{init_1}$", r"Random $q_{init_2}$", r"Random $q_{init_3}$", r"Random $q_{init_4}$"]
b = ["Unconstrained", "Trans blocked along z", "Full trans blocked", "Rot only allowed around z", "Full rot blocked", "Full trans + rot blocked"]
fig, axs = plt.subplots(2, 2, figsize=(14, 8), constrained_layout=True)
for ax, i in zip (axs.flat, range(4)):
    for j, val in enumerate(cm.jet(np.linspace(0,1,6))):
        ax.plot(x,wTotal[i][j], color=val, label=f'{b[j]}')
    
    ax.grid()
    ax.legend()
    ax.set_title('%s' % titles[i], fontsize=16)
    ax.set_ylabel('Manipulability (w)')
    ax.set_xlabel('# Iterations')
    fig.savefig("severities.png", dpi=500)

# Videos of Constraints

In [ ]:
q = list(randomPoseConstrained("upwards"))

In [ ]:
gradientAscent(initialPose=q, constrainedDimensions=[1,1,1,1,1,1])

In [ ]:
print(q)

In [ ]:
gradientAscent(initialPose=q, P=True, constrainedDimensions=[1,1,1,1,1,1])

### Benchmark Jaime

In [8]:
initConfigs = [([-0.66579611, -0.69103315,  2.08672152, -1.95843764, -1.40066813,
        0.45165682, -1.54464071]), ([-0.17749154,  1.37591613, -0.49027116, -1.81571959, -1.78481024,
        1.22836656,  0.41170082]), ([ 0.31976259, -0.22844208,  0.5563295 , -2.70533152, -1.51569737,
        0.57915717, -0.77816797]), ([-0.01545136,  0.67159119,  0.06054268, -2.35163255, -1.69418883,
        0.73834096,  0.06163387]), ([ 0.36272048,  0.86591445,  0.47763373, -1.32017233, -2.66587963,
        0.60918723, -1.57205449]), ([-2.67475968,  1.4564258 ,  0.2880491 , -0.81126216,  2.21712196,
        1.48796303,  0.88667423]), ([ 0.79934447,  0.78169628,  0.42949618, -1.38430286, -2.86879581,
        0.3674066 , -2.0955234 ]), ([-0.26253388,  0.22890602, -0.07043997, -2.23005819, -2.33368897,
        1.16302687, -0.45577531]), ([-0.29290993,  0.41000231, -0.2278887 , -2.84772653, -1.966086  ,
        0.80257395,  0.43056837]), ([ 2.5405737 ,  0.92834202, -2.71444691, -2.03241698, -1.93160033,
        1.99484302, -1.47425825]), ([ 0.45439571, -0.17430196, -0.99887813, -2.15614046, -2.64285093,
        1.52215483, -0.66275443]), ([ 2.84802613,  1.20036492,  2.78548089, -1.00860999, -0.67511501,
        2.53078706,  2.84759646]), ([-2.15960278, -1.13812222,  2.10424103, -0.61158393, -1.53512065,
        1.32972248, -1.71074917]), ([ 0.90905772,  0.5676241 ,  0.03052073, -2.26531808, -0.93938308,
        0.53915868, -0.10678625]), ([ 0.70590751,  0.97772088,  0.55645931, -1.59509076, -2.33548641,
        0.01974743, -1.39632919]), ([-0.06655134, -1.63659092,  0.69205434, -2.64346155, -1.61265196,
        1.71882906, -1.81387732]), ([-0.54005557, -0.26235932,  1.94173304, -2.50814365, -0.87347322,
        0.39518357, -0.63956126]), ([ 1.03026913,  1.59397019, -2.65690107, -2.21262241,  0.10802565,
        3.1346188 , -2.27584502]), ([-2.86529668, -0.26267406, -2.84488514, -1.19071353, -2.54821981,
        1.31096581, -1.81508569]), ([ 0.09934692, -0.43398966, -0.01256874, -2.44908225, -2.31283043,
        1.21243872, -1.0553826 ]), ([ 0.14613208, -0.5310847 ,  0.46615969, -2.7182151 , -1.86455767,
        0.895851  , -1.04959396]), ([ 0.27032353,  0.88252702,  0.32760284, -1.85720281, -1.74172473,
        0.44628582, -0.44447909]), ([ 1.17072191, -1.521212  ,  2.6809015 , -2.45667801, -0.31226422,
        0.15978866, -2.58191468]), ([-0.03452897, -0.58982763,  1.09578764, -2.60057191, -1.48070772,
        0.78345864, -1.18411466]), ([-2.26047379, -0.27311975,  0.05417318, -1.20023271, -1.02928153,
        3.64922106, -1.41523294]), ([ 2.8733891 ,  0.24870227, -0.07408589, -1.33728694,  2.42193012,
        1.30981974,  2.06675681]), ([ 0.60865776, -1.61045803,  2.43041849, -2.03352965, -0.37735326,
        0.10672662, -2.1631851 ]), ([ 0.92458539,  1.09701083, -0.3059522 , -2.2767055 , -0.86008648,
        1.11135596,  0.20893289]), ([ 2.85116887, -0.88517515, -2.66075158, -0.31011113, -2.70279215,
        1.96558838, -1.51547484]), ([-0.2313452 ,  0.83514899,  0.44174693, -2.30856258, -1.58752305,
        0.4456775 ,  0.16540773]), ([ 0.45520614,  1.38600475,  0.58825568, -1.4004542 , -1.71296939,
        0.1052409 , -0.36296822]), ([-2.62570971,  1.05838679, -2.86866927, -1.29444965, -2.21486678,
        2.03331835, -2.82645328]), ([-2.04173204,  0.90534706, -1.2659903 , -2.49770207, -2.84312233,
        0.27392095,  2.1131232 ]), ([-1.03265081,  0.15533867, -0.06337047, -2.91775097, -2.7093328 ,
        0.83552329,  0.21024544]), ([-0.55871752, -1.08826881, -0.30892092, -2.93356083, -2.82397311,
        1.77419322, -0.65843874]), ([ 1.10281654e+00,  2.96671884e-01, -2.79418952e-04, -1.43491830e+00,
       -2.64822881e+00,  7.66769476e-01, -2.16576696e+00]), ([-1.39724392, -0.78635033, -0.02310238, -1.44409901, -0.65489257,
        3.0084881 , -2.35968892]), ([ 1.02782489, -0.44052788, -0.42979588, -2.33660165, -2.52846151,
        0.8515918 , -1.57476431]), ([ 0.12463027,  0.63678701,  0.12449477, -1.7169233 , -2.19145145,
        0.87918182, -0.86207215]), ([-0.29774471,  1.15992333,  0.49044055, -1.96216373, -1.99337255,
        0.35051787, -0.10194112]), ([-0.25379561, -0.76857532, -0.29959491, -2.10524319, -2.5944675 ,
        2.09153725, -1.07702681]), ([-1.01258121, -0.48552366,  0.98266113, -1.94368357, -1.98169201,
        1.61328327, -1.17062656]), ([-0.2308123 ,  0.71019553,  0.09356837, -2.77338971, -1.3650847 ,
        0.71095402,  0.74656252]), ([-0.08570323, -0.61812674, -0.59600411, -1.89242156, -2.7621685 ,
        2.18036547, -0.94727771]), ([-0.28045455, -1.05797152, -0.00704139, -2.38664961, -2.35051423,
        1.95809223, -1.28279563]), ([-0.38331246,  0.42188444, -0.30643385, -1.91461684, -2.46497556,
        1.4543371 , -0.30368305]), ([ 0.00604438,  0.40810116,  1.20219557, -2.29704646, -1.48415863,
       -0.00494078, -0.68997927]), ([-0.84410464, -0.72537393, -0.2443885 , -1.12827014, -0.74898421,
        2.80500095, -2.67775349]), ([-0.60520207,  1.11754829, -1.07250839, -0.97437243, -2.11676127,
        2.22370725,  0.08612538]), ([ 0.09231936, -1.61499901,  0.8135256 , -2.42600274, -1.57389323,
        1.55243752, -2.12766381])]

In [9]:
import time
from scipy.spatial.transform import Rotation as R


In [10]:
TOTAL = []
setupList = [[4,5000,0.00001],[4,10000,0.0001],[4,20000,0.0001],[6.5, 5000, 0.00001],[6.5, 10000, 0.0001],[6.5, 20000, 0.0001],[9,5000,0.0001],[9,10000,0.0001],[9,20000,0.0001]]
for setup in setupList:
    timeSpans =[] #shape : 50
    M = [] #shape : 50 #in seconds
    rotZ = [] #shape : 50 # absolute change in radians
    for config in initConfigs:
        start_time = time.time()
        r = R.from_matrix(kdl_kin.forward(config)[:3,0:3])
        rotvec = r.as_rotvec()
        m = gradientAscent(lr = setup[0], constrainedDimensions = [0,0,0,1,1,0], P=True, initialPose=config, k=setup[1], c=setup[2]) 
        print(m[0][-1])
        rr = R.from_matrix(kdl_kin.forward(m[1].tolist()[:7])[:3,0:3])
        newRotVec = rr.as_rotvec()
        timeSpans.append(time.time() - start_time)
        M.append(m[0][-1])
        rotZ.append(abs(rotvec-newRotVec))
    average_time = sum(timeSpans)/ len(timeSpans)
    average_manip_obtained = sum(M)/ len(M)
    average_deviation = sum(rotZ)/len(rotZ)  #in radians
    print("---------------------------------------------------------------------\n")
    print(f'HPs : lr = {setup[0]}, k = {setup[1]}, c = {setup[2]}')
    print("---------------------------------------------------------------------")
    print("Average Time : ",average_time, "sec")
    print("Average Max Manipulability : ",average_manip_obtained.item()," (", (torch.round(average_manip_obtained.item()/np.max(M)*100)).item(), "% of max. manip)")
    print("Average Deviation around x (in radians) : ",average_deviation[0])
    print("Average Deviation around y (in radians) : ",average_deviation[1])
    print("---------------------------------------------------------------------\n")
    

tensor(0.0095, grad_fn=<MulBackward0>)
tensor(0.0137, grad_fn=<MulBackward0>)
tensor(0.0135, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(0.0002, grad_fn=<MulBackward0>)
tensor(5.4926e-05, grad_fn=<MulBackward0>)
tensor(0.0135, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(0.0002, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(5.4263e-07, grad_fn=<MulBackward0>)
tensor(0.0092, grad_fn=<MulBackward0>)
tensor(0.0134, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(0.0095, grad_fn=<MulBackward0>)
tensor(0.0094, grad_fn=<MulBackward0>)
tensor(0.0001, grad_fn=<MulBackward0>)
tensor(1.8844e-06, grad_fn=<MulBackward0>)
tensor(0.0076, grad_fn=<MulBackward0>)
tensor(0.0074, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(1.0828e-05, grad_fn=<MulBackward0>)
tensor(0.0096, grad_fn=<MulBackward0>)
tensor(8.2271e-05, grad_fn=<MulBackward0>)
tenso

tensor(0.0075, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(1.0840e-05, grad_fn=<MulBackward0>)
tensor(0.0096, grad_fn=<MulBackward0>)
tensor(8.2717e-05, grad_fn=<MulBackward0>)
tensor(0.0124, grad_fn=<MulBackward0>)
tensor(4.4146e-05, grad_fn=<MulBackward0>)
tensor(0.0135, grad_fn=<MulBackward0>)
tensor(5.1930e-07, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(9.8414e-07, grad_fn=<MulBackward0>)
tensor(1.4902e-05, grad_fn=<MulBackward0>)
tensor(0.0001, grad_fn=<MulBackward0>)
tensor(5.1773e-05, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(0.0095, grad_fn=<MulBackward0>)
tensor(0.0136, grad_fn=<MulBackward0>)
tensor(0.0137, grad_fn=<MulBackward0>)
tensor(0.0137, grad_fn=<MulBackward0>)
tensor(0.0076, grad_fn=<MulBackward0>)
tensor(0.0095, grad_fn=<MulBackward0>)
tensor(0.0137, grad_fn=<MulBackward0>)
tensor(0.0077, grad_fn=<MulBackward0>)
tensor(0.0077, grad_fn=<MulBackward0

tensor(0.0134, grad_fn=<MulBackward0>)
tensor(0.0073, grad_fn=<MulBackward0>)
tensor(0.0069, grad_fn=<MulBackward0>)
tensor(0.0131, grad_fn=<MulBackward0>)
tensor(0.0074, grad_fn=<MulBackward0>)
tensor(0.0075, grad_fn=<MulBackward0>)
tensor(0.0130, grad_fn=<MulBackward0>)
tensor(2.0662e-05, grad_fn=<MulBackward0>)
tensor(0.0009, grad_fn=<MulBackward0>)
tensor(0.0132, grad_fn=<MulBackward0>)
tensor(0.0089, grad_fn=<MulBackward0>)
---------------------------------------------------------------------

HPs : lr = 9, k = 5000, c = 0.0001
---------------------------------------------------------------------
Average Time :  5.745791373252868 sec
Average Max Manipulability :  0.0059394692070782185  ( 44.0 % of max. manip)
Average Deviation around x (in radians) :  0.00469019252762448
Average Deviation around y (in radians) :  0.004094024995405102
---------------------------------------------------------------------

tensor(0.0180, grad_fn=<MulBackward0>)
tensor(0.0262, grad_fn=<MulBackward0>)


In [ ]:
average_time = sum(timeSpans)/ len(timeSpans)
average_manip_obtained = sum(M)/ len(M)
average_deviation = sum(rotZ)/len(rotZ)  #in radians


In [ ]:
print('HPs : lr = 9, k = 20.000, c = 0.0001 ')
print("---------------------------------------------------------------------")
print("Average Time : ",average_time, "sec")
print("Average Max Manipulability : ", (torch.round(average_manip_obtained.item()/np.max(M)*100)).item(), "%")
print("Average Deviation around x (in radians) : ",average_deviation[0])
print("Average Deviation around y (in radians) : ",average_deviation[1])